### Extracción de datos de la API oficial web "Spotify Web API"  y de la API oficial de Last.fm " Last.fm API "

#### Instalación de librerías

In [1]:
import sys
print(sys.executable)

c:\Users\apdue\anaconda3\python.exe


In [2]:
import sys
!{sys.executable} -m pip install spotipy

In [3]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
!{sys.executable} -m pip install mysql-connector-python

#### Extracción de datos de la API oficial web "Spotify Web API" 

In [5]:
# Importamos Librerías
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import time

# SPOTIFY 
CLIENT_ID = "ce410d235c6444c9928df476e4270803"
CLIENT_SECRET = "af63ec6e05084a4d9edc175a055bd4f2"

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

# PARÁMETROS 
generos = ['pop', 'rock', 'jazz', 'reggaeton']
anio_inicio, anio_final = 2019, 2024
max_canciones = 500
todos_los_resultados = []

# RECOLECCIÓN DE CANCIONES 
for genero in generos:
    print(f"Buscando canciones para el género {genero}...")
    canciones_obtenidas = 0
    offset = 0
    limite = 50

    while canciones_obtenidas < max_canciones:
        query = f'genre:"{genero}" year:{anio_inicio}-{anio_final}'
        respuesta = sp.search(q=query, type='track', limit=limite, offset=offset, market="ES")
        tracks = respuesta.get('tracks', {}).get('items', [])

        if not tracks:
            break

        for track in tracks:
            todos_los_resultados.append({
                'name_track': track['name'],
                'name_artist': track['artists'][0]['name'],
                'genre': genero,
                'type': track['type'],
                'year': track['album']['release_date'][:4]
            })

        canciones_obtenidas += len(tracks)
        offset += limite
        time.sleep(0.2)

    print(f"Se obtuvieron {canciones_obtenidas} canciones para {genero}")

print(f"Total de canciones obtenidas: {len(todos_los_resultados)}")

# CONVERTIR A DATAFRAME 
df_spotipy = pd.DataFrame(todos_los_resultados)

#  MYSQL 
user = 'root'
password = 'AlumnaAdalab'
host = '127.0.0.1'
db_name = 'music_stream_proyecto'

# Crear la base de datos si no existe
try:
    cnx = mysql.connector.connect(user=user, password=password, host=host)
    cursor = cnx.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Base de datos '{db_name}' creada o ya existente.")
    cursor.close()
    cnx.close()
except mysql.connector.Error as err:
    print(f"Ocurrió un error creando la base de datos: {err}")

# Conectar con SQLAlchemy para pasar DataFrame directamente
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{db_name}')

# Guardar DataFrame en MySQL
try:
    df_spotipy.to_sql('canciones', engine, if_exists='replace', index=False)
    print(f"Se insertaron {len(df_spotipy)} canciones en la tabla 'canciones'.")
except Exception as e:
    print(f"Ocurrió un error al guardar en SQL: {e}")

Buscando canciones para el género pop...
Se obtuvieron 500 canciones para pop
Buscando canciones para el género rock...
Se obtuvieron 500 canciones para rock
Buscando canciones para el género jazz...
Se obtuvieron 500 canciones para jazz
Buscando canciones para el género reggaeton...
Se obtuvieron 113 canciones para reggaeton
Total de canciones obtenidas: 1613
Base de datos 'music_stream_proyecto' creada o ya existente.
Se insertaron 1613 canciones en la tabla 'canciones'.


API LAST.FM 

In [ ]:
import requests
import pandas as pd
import re
import time

# ---- Lista de artistas únicos desde Spotify ----
lista_artistas = df_spotipy['name_artist'].dropna().unique().tolist()

# ---- Función para limpiar nombres de artistas ----
def limpiar_nombre_artista(nombre):
    nombre = nombre.lower()
    nombre = re.sub(r"\(.*?\)|feat\. .*|ft\. .*", "", nombre)  # quitar feat, ft, paréntesis
    nombre = re.sub(r"[^a-z0-9\s]", "", nombre)  # quitar caracteres especiales
    nombre = nombre.strip()
    return nombre

# Lista limpia de artistas
lista_artistas_limpia = [limpiar_nombre_artista(a) for a in lista_artistas]

# ---- Configuración Last.fm ----
API_KEY = "4c376bffe433789718d87658d629efca"
BASE_URL = "http://ws.audioscrobbler.com/2.0/"

def obtener_datos_lastfm_artistas(lista_artistas, max_tracks=5, max_similares=5, sleep_time=0.2):
    registro_artistas = []

    for artista_nombre in lista_artistas:
        try:
            # --- Información general del artista ---
            resp_info = requests.get(BASE_URL, params={
                "method": "artist.getinfo",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json"
            }, timeout=10)
            resp_info.raise_for_status()
            datos = resp_info.json().get("artist")
            if not datos:
                continue

            stats = datos.get("stats", {})
            bio = datos.get("bio", {}).get("summary", "")
            url = datos.get("url", "")
            tags_list = datos.get("tags", {}).get("tag", [])
            if isinstance(tags_list, dict):
                tags_list = [tags_list]
            tags = [t.get("name","") for t in tags_list]

            # --- Top tracks ---
            top_tracks_info = []
            resp_tracks = requests.get(BASE_URL, params={
                "method": "artist.gettoptracks",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json",
                "limit": max_tracks
            }, timeout=10)
            resp_tracks.raise_for_status()
            tracks_json = resp_tracks.json().get("toptracks", {}).get("track", [])
            if isinstance(tracks_json, dict):
                tracks_json = [tracks_json]

            for track in tracks_json:
                track_name = track.get("name","")
                track_info = {"name": track_name}
                try:
                    resp_track = requests.get(BASE_URL, params={
                        "method": "track.getinfo",
                        "artist": artista_nombre,
                        "track": track_name,
                        "api_key": API_KEY,
                        "autocorrect": 1,
                        "format": "json"
                    }, timeout=10)
                    resp_track.raise_for_status()
                    track_data = resp_track.json().get("track", {})
                    tags_track = track_data.get("toptags", {}).get("tag", [])
                    if isinstance(tags_track, dict):
                        tags_track = [tags_track]
                    tags_cancion = [tg.get("name","") for tg in tags_track]
                    track_info.update({
                        "album": track_data.get("album", {}).get("title",""),
                        "duration_s": int(track_data.get("duration") or 0)//1000,
                        "tags": tags_cancion,
                        "wiki": track_data.get("wiki", {}).get("summary","")
                    })
                except:
                    pass
                top_tracks_info.append(track_info)

            # --- Artistas similares ---
            similares = []
            resp_sim = requests.get(BASE_URL, params={
                "method": "artist.getsimilar",
                "artist": artista_nombre,
                "api_key": API_KEY,
                "autocorrect": 1,
                "format": "json",
                "limit": max_similares
            }, timeout=10)
            resp_sim.raise_for_status()
            similar_json = resp_sim.json().get("similarartists", {}).get("artist", [])
            if isinstance(similar_json, dict):
                similar_json = [similar_json]
            similares = [s.get("name","") for s in similar_json]

            # --- Guardar info del artista ---
            registro_artistas.append({
                "Artist": artista_nombre,
                "Listeners": int(stats.get("listeners",0) or 0),
                "Playcount": int(stats.get("playcount",0) or 0),
                "URL": url,
                "Biography": bio,
                "Tags": tags,
                "TopTracks": top_tracks_info,
                "SimilarArtists": similares
            })

            time.sleep(sleep_time)  # respetar rate limit

        except Exception as e:
            print(f"Error con artista {artista_nombre}: {e}")
            continue

    return pd.DataFrame(registro_artistas)

Hemos tenido que utilizar concurrencia de hilos para poder ejecutar el código, pues se hacía muy lento

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Función que procesa un solo artista (envoltorio)
def obtener_datos_lastfm_artista(artista):
    return obtener_datos_lastfm_artistas([artista], max_tracks=5, max_similares=5, sleep_time=0)

# ---- Ejecutar en paralelo ----
resultados = []
with ThreadPoolExecutor(max_workers=10) as executor:  # prueba 10-15 para más velocidad
    futures = {executor.submit(obtener_datos_lastfm_artista, artista): artista for artista in lista_artistas_limpia}
    
    for future in as_completed(futures):
        artista = futures[future]
        try:
            df_tmp = future.result()
            if not df_tmp.empty:
                resultados.append(df_tmp)
        except Exception as e:
            print(f"Error procesando artista {artista}: {e}")

# ---- Unir todos los resultados ----
df_lastfm = pd.concat(resultados, ignore_index=True) if resultados else pd.DataFrame()


        Artist  Listeners  Playcount                                    URL  \
0   elton john    4965352  152014971   https://www.last.fm/music/Elton+John   
1     dua lipa    3038687  293563049     https://www.last.fm/music/Dua+Lipa   
2  teddy swims     995058   16831289  https://www.last.fm/music/Teddy+Swims   
3       loreen     750933   19243499       https://www.last.fm/music/Loreen   
4          djo    1852063   62255211          https://www.last.fm/music/Djo   

                                           Biography  \
0  Sir Elton Hercules John (born Reginald Kenneth...   
1  Dua Lipa (born 22 August 1995) is an English, ...   
2  Jaten Dimsdale, known by his moniker, Teddy Sw...   
3  Loreen (born Lorine Zineb Nora Talhaoui on Oct...   
4  Djo is the musical moniker of American actor/m...   

                                                Tags  \
0  [pop, classic rock, singer-songwriter, rock, p...   
1  [pop, synthpop, electropop, british, female vo...   
2      [soul, singer

In [8]:
# Importamos las librerías necesarias para la conexión Python-Sql
import json
from sqlalchemy import create_engine, VARCHAR, BIGINT, TEXT
import mysql.connector

# Parámetros de conexión
user = 'root'
password = 'AlumnaAdalab'
host = '127.0.0.1'
db_name = 'music_stream_proyecto'

# ---- Crear la base de datos si no existe ----
try:
    cnx = mysql.connector.connect(user=user, password=password, host=host)
    cursor = cnx.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    print(f"Base de datos '{db_name}' creada o ya existente.")
    cursor.close()
    cnx.close()
except mysql.connector.Error as err:
    print(f"Ocurrió un error creando la base de datos: {err}")

# ---- Convertir listas/diccionarios a JSON strings ----
df_lastfm_copy = df_lastfm.copy()
for col in ['Tags', 'TopTracks', 'SimilarArtists']:
    df_lastfm_copy[col] = df_lastfm_copy[col].apply(lambda x: json.dumps(x, ensure_ascii=False))

# ---- Conexión a MySQL usando SQLAlchemy ----
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}/{db_name}')

# ---- Guardar DataFrame en SQL ----
try:
    df_lastfm_copy.to_sql(
        'artistas',
        engine,
        if_exists='replace',
        index=False,
        dtype={
            'Artist': VARCHAR(100),
            'Listeners': BIGINT,
            'Playcount': BIGINT,
            'URL': VARCHAR(255),
            'Biography': TEXT,
            'Tags': TEXT,
            'TopTracks': TEXT,
            'SimilarArtists': TEXT
        }
    )
    print("Datos guardados correctamente en la tabla 'artistas'.")
except Exception as e:
    print(f"Ocurrió un error al guardar en SQL: {e}")


Base de datos 'music_stream_proyecto' creada o ya existente.
Datos guardados correctamente en la tabla 'artistas'.


Guardamos los datos en csv

In [ ]:

# Guardar DataFrame de canciones de Spotify
df_spotipy.to_csv('canciones_spotify.csv', index=False, encoding='utf-8-sig')
print(f"Se guardaron {len(df_spotipy)} canciones en 'canciones_spotify.csv'.")

# Guardar DataFrame de artistas de Last.fm
df_lastfm_copy.to_csv('artistas_lastfm.csv', index=False, encoding='utf-8-sig')
print(f"Se guardaron {len(df_lastfm_copy)} artistas en 'artistas_lastfm.csv'.")

Se guardaron 1613 canciones en 'canciones_spotify.csv'.
Se guardaron 187 artistas en 'artistas_lastfm.csv'.


COMPARACIÓN SCRIPTS DE PYTHON  Y DOCUMENTACIÓN APIS

| Aspecto implementado | Ajuste con la documentación oficial de SPOTIPY |
|-----------------------|--------------------------------------|
| Autenticación mediante credenciales de cliente. | Se ajusta al flujo de *Client Credentials* descrito en la documentación. |
| Creación de una instancia del cliente de Spotify con un administrador de autenticación. | Coincide con el procedimiento oficial recomendado (`Spotify(auth_manager=...)`). |
| Uso de consultas avanzadas con filtros de género y rango de años. | Se ajusta a la sintaxis de búsqueda avanzada soportada por la API de Spotify, que permite combinar múltiples criterios. |
| Ejecución de búsquedas de pistas especificando tipo de recurso, límite, offset y mercado. | Coincide con los parámetros documentados para el método `search`. |
| Extracción de información relevante de cada pista (nombre, artista y año de lanzamiento). | Coincide con la estructura de datos proporcionada por la API, que incluye estos campos en los resultados. |
| Inclusión de un retardo entre peticiones para evitar sobrecargar la API. | No aparece en la documentación oficial, pero es considerado una buena práctica para respetar las políticas de uso y evitar bloqueos por exceso de solicitudes. |
| Conversión de los resultados a un DataFrame para su análisis. | No forma parte de la documentación de Spotipy, ya que la librería devuelve datos en formato JSON, pero se utiliza para análisis posteriores. |

| Aspecto implementado | Ajuste con la documentación oficial de Last.fm |
|-----------------------|-----------------------------------------------|
| Uso de `artist.getinfo` para obtener información de un artista. | Coincide con la documentación; `artist.getinfo` retorna detalles del artista, estadísticas, etiquetas y enlaces. |
| Inclusión de parámetros `api_key`, `autocorrect=1` y `format=json`. | Coincide  con lo documentado: `api_key` obligatorio, `autocorrect` opcional y `format=json` para obtener la respuesta en JSON. |
| Extracción de estadísticas (`listeners`, `playcount`), biografía (`bio.summary`), URL, y etiquetas (`tags.tag`). | Coincide con la estructura de datos devuelta según la doc de `artist.getinfo`. |
| Manejo de top tracks y artistas similares (`toptracks.track`, `similar.artist`). | Coincide con la documentación; los objetos pueden ser listas o diccionarios si hay un solo elemento. |
| Límite de resultados (`max_tracks`, `max_similares`). | No es parte de la API oficial; es un control implementado para limitar la cantidad de datos procesados. |
| Manejo de errores con `try/except` y `resp.raise_for_status()`. | No aparece en la documentación, pero se implementa para evitar fallos por problemas de red o respuestas inválidas. |
| Ejecución paralela de solicitudes con `ThreadPoolExecutor` y `as_completed`. | No está documentado en la API oficial; no obstante, se ha utilizado para optimizar rendimiento al consultar varios artistas. |
| Conversión final de los resultados a un `DataFrame` de pandas. | La documentación de Last.fm devuelve JSON; pero transformamos a DataFrame para posterior análisis de datos. |

COMPARACIÓN EN FORMA DE EXTRACCIÓN DE DATOS PARA CADA API

| Aspecto | Spotipy (Spotify) | Last.fm |
|---------|-----------------|---------|
| Autenticación | Se usa `SpotifyClientCredentials` con `client_id` y `client_secret`. Coincide con la documentación oficial. | Se usa `api_key` en cada solicitud, como indica la documentación. |
| Instanciación del cliente | Se crea objeto `Spotify(auth_manager=...)`. Coincide con la doc. | No aplica, se realizan llamadas directas a la URL de la API con `requests`. |
| Método de consulta principal | `sp.search` con filtros avanzados (`genre`, `year`), `type`, `limit`, `offset`, `market`. | `artist.getinfo` con parámetros (`artist`, `autocorrect`, `format`). |
| Extracción de datos | Campos como `name`, `artists`, `album.release_date`. | Campos como `listeners`, `playcount`, `bio.summary`, `tags.tag`, `toptracks.track`, `similar.artist`. |
| Manejo de paginación | Sí, usando `limit` y `offset` en `sp.search`. Posible mejora: usar `next` de la API. | Parcial: No hay paginación automática en `artist.getinfo`. Se limita manualmente con `max_tracks` y `max_similares`. |
| Control de resultados | Controlado con `max_canciones` por género. | Controlado con `max_tracks` y `max_similares` por artista. |
| Manejo de errores | Implícito; la librería lanza excepciones (`SSLError`, `ConnectionError`). | Explícito con `try/except` y `resp.raise_for_status()`. |
| Optimización / paralelización | No implementada. | Paralelización con `ThreadPoolExecutor` para múltiples artistas. |
| Transformación de datos | Conversión a `pandas.DataFrame` para análisis. | Conversión a `pandas.DataFrame` para análisis. |
| Buenas prácticas adicionales | `time.sleep` para evitar *rate limiting*. | Manejo de estructuras variables en JSON y límite de resultados para evitar sobrecarga. |